In [49]:

from PIL import Image
import os
from collections import defaultdict
import random
import json
random.seed(42)


def resize_pad(image, size):
    """
    Resize and pad an image to a square image of size size
    """
    # Resize
    width, height = image.size
    if width > height:
        image = image.resize((size, int(height * size / width)))
    else:
        image = image.resize((int(width * size / height), size))

    #make the image black and white, not grayscale
    image = image.convert('L')
    # Pad
    new_image = Image.new('L', (size, size), 255)
    new_image.paste(image, ((size - image.size[0]) // 2, (size - image.size[1]) // 2))
    return new_image

In [50]:

SRC_DIR = "base_for_fewshot"
DST_DIR = "FEW_SHOTS"

for filename in os.listdir(DST_DIR):
    file_path = os.path.join(DST_DIR, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)

    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [51]:
global_hashes = set()

def make_combinations(folder1, folder2, dst_dir, priority = False):

    if "/real/" in folder1:
        folder1_type = "real"
    else:
        folder1_type = "impossible"
    
    if "/real/" in folder2:
        folder2_type = "real"
    else:
        folder2_type = "impossible"

    i = len(os.listdir(dst_dir))
    for filename1 in os.listdir(folder1):
        for filename2 in os.listdir(folder2):
            file_path1 = os.path.join(folder1, filename1)
            file_path2 = os.path.join(folder2, filename2)
            
            if file_path1 == file_path2:
                continue
            
            img1 = Image.open(file_path1)
            img2 = Image.open(file_path2)
            img1 = resize_pad(img1, 256)
            img2 = resize_pad(img2, 256)
            
            new_img = Image.new('L', (512, 256))
            new_img.paste(img1, (0, 0))
            new_img.paste(img2, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)

            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}_p.jpg"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}.jpg"))
            i += 1

            new_img = Image.new('L', (512, 256))
            new_img.paste(img2, (0, 0))
            new_img.paste(img1, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)
            
            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}_p.jpg"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}.jpg"))
            i += 1


In [52]:
types = [1]

for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_fewshot/real/{t1}", f"base_for_fewshot/impossible/{t2}", DST_DIR, t1==t2)

len(os.listdir(DST_DIR))


8

In [53]:
for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_fewshot/real/{t1}", f"base_for_fewshot/real/{t2}", DST_DIR, t1==t2)
        
len(os.listdir(DST_DIR))


10

In [54]:
for t1 in types:
    for t2 in types:
        make_combinations(f"base_for_fewshot/impossible/{t1}", f"base_for_fewshot/impossible/{t2}", DST_DIR, t1==t2)
        
len(os.listdir(DST_DIR))

12

In [55]:

d = defaultdict(list)

for filename in os.listdir(DST_DIR):
    x = filename.split(".")[0][5:]

    d[x].append(filename)



In [58]:
final_pick = []

for k in d:
    print(k, len(d[k]))
    final_pick.append(random.choice(d[k]))

real_impossible_p 4
impossible_real_p 4
real_real_p 2
impossible_impossible_p 2


In [59]:
final_pick

['0000_real_impossible_p.jpg',
 '0001_impossible_real_p.jpg',
 '0009_real_real_p.jpg',
 '0010_impossible_impossible_p.jpg']

In [60]:
question = "Which object is geometrically impossible?"
options = ["The left object", "The right object", "Both objects", "Neither object"]
# options = ["Neither object", "The right object", "The left object", "Both objects"]
# [
#   {
#     "question": "Are the two birds the same color?",
#     "options": [
#         "no",
#         "yes",
#         "cannot tell from the image"
#     ],
#     "answer": "yes",
#     "image": "110.jpg",
#     "category": "color"
#   }
# ]

for_json = []
for i, filename in enumerate(final_pick):
    d = {}
    d["question"] = question
    d["options"] = options

    if "real_real" in filename:
        d["answer"] = "Neither object"
    elif "real_impossible" in filename:
        d["answer"] = "The right object"
    elif "impossible_real" in filename:
        d["answer"] = "The left object"
    elif "impossible_impossible" in filename:
        d["answer"] = "Both objects"

    d["image"] = filename
    d["category"] = "soft_localization"
    for_json.append(d)

with open('fewshot_labels.json', 'w') as f:
    json.dump(for_json, f, indent=2)